In [2]:
import pandas as pd
import datetime
from datetime import timedelta
import requests
import plotly as py
import plotly.express as px
import chart_studio.tools as tls

In [3]:
#BUILDS DATAFRAME FOR COMMUNITY DATA
columns = ['id', 'symbol', 'price', 'mkt_cap', 'vol', 'reddit_posts_48h', 'reddit_comments_48h', 'reddit accounts_48h', 'forks', 'stars', 'contributors', 'twitter_followers', 'alexa_rank']
df = pd.DataFrame(columns = columns)
# display(df)
today = datetime.datetime.now()
yesterday = today -timedelta(days = 2)
yesterday = yesterday.strftime("%d-%m-%Y")
# date = str(yesterday)
date = yesterday
# print('yesterday', yesterday)
# print('date', date)

def coinFunc(currentCoin, date): 
    coin= currentCoin
    geckoReq = 'https://api.coingecko.com/api/v3/coins/'+coin+'/history?date='+date
    priceReq = 'https://api.coingecko.com/api/v3/simple/price?ids=' + coin + '&vs_currencies=usd&include_market_cap=true&include_24hr_vol=true'
    price = requests.get(priceReq).json()
    r =requests.get(geckoReq).json()
#     display(r)
    #     print(r)
    coin_id = r['id']
    coin_symbol = r['symbol']

    #price data
    current_price = price[currentCoin]['usd']
    mkt_cap = price[currentCoin]['usd_market_cap']
    vol = price[currentCoin]['usd_24h_vol']

    #reddit data
    reddit_average_posts_48h = r['community_data']['reddit_average_posts_48h']
    reddit_average_comments_48h = r['community_data']['reddit_average_comments_48h']
    reddit_accounts_active_48h = r['community_data']['reddit_accounts_active_48h']
    #github data
    forks = r['developer_data']['forks']
    stars = r['developer_data']['stars']
    contributors = r['developer_data']['pull_request_contributors']
    twitter_followers = r['community_data']['twitter_followers']
    alexa_rank = r['public_interest_stats']['alexa_rank']


    dataInsert = [coin_id, coin_symbol, current_price, mkt_cap, vol, reddit_average_posts_48h, reddit_average_comments_48h, reddit_accounts_active_48h, forks, stars, contributors, twitter_followers, alexa_rank]
    # print(dataInsert)

    df_length = len(df)
    df.loc[df_length] = dataInsert
    
# coinFunc('audius', yesterday)    
# display(df)
# print(r)
# print(r['developer_data'])
# print(r['community_data'])
# print(coin_price)

theCoins = ['ethereum', 'bitcoin', 'binancecoin', 'solana', 'polkadot', 'flow', 'dogecoin', 'audius']
trendingReq = 'https://api.coingecko.com/api/v3/search/trending'
trends =requests.get(trendingReq).json()
# print(r)
trending = []
#could be for 7 coins but lets just do top 5
for i in range(5):
    trendingCoin = trends['coins'][i]['item']['id']
    trending.append(trendingCoin)

theCoins.extend(trending)

for crypto in theCoins:
    coinFunc(crypto, yesterday)
    
display(df)
myCoinsList = df['id'].tolist()
print(myCoinsList)

,id,symbol,price,mkt_cap,vol,reddit_posts_48h,reddit_comments_48h,reddit accounts_48h,forks,stars,contributors,twitter_followers,alexa_rank
0,ethereum,eth,4129.150000,4.891626e+11,2.565133e+10,8.000,519.182,1278.25,12496,33825,597,1920252,3541
1,bitcoin,btc,48633.000000,9.220995e+11,3.347927e+10,7.091,1191.182,3936.25,30725,59776,760,3896601,6000
2,binancecoin,bnb,550.830000,9.302309e+10,2.102688e+09,9.167,243.333,737.153846153846,2034,3550,26,6446615,109
3,solana,sol,187.750000,5.791693e+10,3.180264e+09,8.727,134.091,457.333333333333,1153,5867,212,1053249,3321
4,polkadot,dot,27.430000,2.926171e+10,1.259529e+09,2.100,23.700,88.3636363636364,1012,4827,143,967622,13319
5,flow,flow,9.400000,2.954552e+09,6.412195e+07,0.000,0.000,None,None,None,None,121836,221603
6,dogecoin,doge,0.167663,2.236066e+10,1.369121e+09,8.333,1037.583,2003.53846153846,2383,13531,126,2600868,33845
7,audius,audio,1.700000,8.696229e+08,2.628305e+07,1.750,10.583,28.2307692307692,None,None,None,111588,21764
8,fantom,ftm,1.500000,3.840111e+09,4.414136e+08,4.000,45.600,122.818181818182,52,163,23,258132,16767
9,brokoli,brkl,1.350000,2.331334e+07,9.453284e+05,0.000,0.000,None,None,None,None,None,182185


['ethereum', 'bitcoin', 'binancecoin', 'solana', 'polkadot', 'flow', 'dogecoin', 'audius', 'fantom', 'brokoli', 'unix', 'wonderland', 'terra-luna']


In [4]:
### BUILDING REDDIT COMMENTS HTML GRAPH
display(df.head(5))

fig = px.histogram(df, x=df['id'], y=df['reddit_comments_48h'],
                labels={
                     "sum of reddit_comments_48h": "Reddit Comments 48h",
                     "id": "token"
#                      "species": "Species of Iris"
                 },)
py.offline.plot(fig, filename='geckoRedditComments.html')


# print(tls.get_embed('https://plotly.com/~fooba/1'))
fig.show()



,id,symbol,price,mkt_cap,vol,reddit_posts_48h,reddit_comments_48h,reddit accounts_48h,forks,stars,contributors,twitter_followers,alexa_rank
0,ethereum,eth,4129.15,4.891626e+11,2.565133e+10,8.000,519.182,1278.25,12496,33825,597,1920252,3541
1,bitcoin,btc,48633.00,9.220995e+11,3.347927e+10,7.091,1191.182,3936.25,30725,59776,760,3896601,6000
2,binancecoin,bnb,550.83,9.302309e+10,2.102688e+09,9.167,243.333,737.153846153846,2034,3550,26,6446615,109
3,solana,sol,187.75,5.791693e+10,3.180264e+09,8.727,134.091,457.333333333333,1153,5867,212,1053249,3321
4,polkadot,dot,27.43,2.926171e+10,1.259529e+09,2.100,23.700,88.3636363636364,1012,4827,143,967622,13319


In [9]:
### BUILDING REDDIT COMMENTS HTML GRAPH
display(df.head(5))

fig = px.scatter(df, x=df['id'], y=df['twitter_followers'],
                labels={
                     "sum of twitter_followers": "twitter_followers",
                     "id": "token"
#                      "species": "Species of Iris"
                 },)
py.offline.plot(fig, filename='twitter_followers.html')


# print(tls.get_embed('https://plotly.com/~fooba/1'))
fig.show()


,id,symbol,price,mkt_cap,vol,reddit_posts_48h,reddit_comments_48h,reddit accounts_48h,forks,stars,contributors,twitter_followers,alexa_rank
0,ethereum,eth,4129.15,4.891626e+11,2.565133e+10,8.000,519.182,1278.25,12496,33825,597,1920252,3541
1,bitcoin,btc,48633.00,9.220995e+11,3.347927e+10,7.091,1191.182,3936.25,30725,59776,760,3896601,6000
2,binancecoin,bnb,550.83,9.302309e+10,2.102688e+09,9.167,243.333,737.153846153846,2034,3550,26,6446615,109
3,solana,sol,187.75,5.791693e+10,3.180264e+09,8.727,134.091,457.333333333333,1153,5867,212,1053249,3321
4,polkadot,dot,27.43,2.926171e+10,1.259529e+09,2.100,23.700,88.3636363636364,1012,4827,143,967622,13319


In [6]:
#clean version of historic prices request
#timestamps are in day-month-year
import requests
import datetime
import pandas as pd
from datetime import timedelta

# dfPrices = pd.DataFrame()
# geckoReq = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days=60&interval=daily'
geckoReq = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days=60&interval=daily'

r = requests.get(geckoReq).json()
ts = r['prices'][0][0]
ts = ts/1000
HistPricesList = []
for i in range(len(r['prices'])):
    currentUnix = r['prices'][i][0] 
    price = r['prices'][i][1]
    currentUnix = currentUnix/1000
    currentTS = datetime.datetime.fromtimestamp(currentUnix).strftime("%d-%m-%Y")
    HistPricesList.append([currentTS])
global dfPrices
dfPrices = pd.DataFrame(HistPricesList, columns = ['dates'])
print(dfPrices)

# display(dfCoin)
# print('dfprices')
# display(dfPrices)

# coin='ethereum'
def historicalCoinPrices(coin):
    global dfPrices
    # geckoReq = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days=60&interval=daily'
    geckoReq = 'https://api.coingecko.com/api/v3/coins/'+coin+'/market_chart?vs_currency=usd&days=60&interval=daily'

    r = requests.get(geckoReq).json()
    ts = r['prices'][0][0]
    # print(ts)
    ts = ts/1000
    print(datetime.datetime.fromtimestamp(ts).strftime("%d-%m-%Y"))
    # print('prices length',len(r['prices']))
    HistPricesList = []
    for i in range(len(r['prices'])):
        currentUnix = r['prices'][i][0] 
        price = r['prices'][i][1]
        currentUnix = currentUnix/1000
        currentTS = datetime.datetime.fromtimestamp(currentUnix).strftime("%d-%m-%Y")
    #     print('price: ', price, 'TS: ', currentTS)
        HistPricesList.append([currentTS, price])
    #     currentUnix = currentTS
#     print(HistPricesList)
    dfCoin = pd.DataFrame(HistPricesList, columns = ['dates', coin])
    # display(dfCoin)
    dfPrices = pd.merge(dfPrices, dfCoin[coin], left_on = dfPrices['dates'], right_on=dfCoin['dates']).drop(['key_0'], axis = 1)
    # display(dfPrices)

for i in myCoinsList:
    historicalCoinPrices(i)
    print('just added: ', i)


display(dfPrices.head(5))

         dates
0   06-10-2021
1   07-10-2021
2   08-10-2021
3   09-10-2021
4   10-10-2021
..         ...
56  01-12-2021
57  02-12-2021
58  03-12-2021
59  04-12-2021
60  05-12-2021

[61 rows x 1 columns]
06-10-2021
just added:  ethereum
06-10-2021
just added:  bitcoin
06-10-2021
just added:  binancecoin
06-10-2021
just added:  solana
06-10-2021
just added:  polkadot
06-10-2021
just added:  flow
06-10-2021
just added:  dogecoin
06-10-2021
just added:  audius
06-10-2021
just added:  fantom
06-10-2021
just added:  brokoli
27-11-2021
just added:  unix
06-10-2021
just added:  wonderland
06-10-2021
just added:  terra-luna


,dates,ethereum,bitcoin,binancecoin,solana,polkadot,flow,dogecoin,audius,fantom,brokoli,unix,wonderland,terra-luna
0,27-11-2021,4084.088486,54533.403882,598.065883,192.136200,34.951339,11.821434,0.204382,2.411792,2.067045,1.270884,1.028047,7445.431029,43.446797
1,28-11-2021,4290.091863,57237.661137,611.448234,200.461419,35.644429,12.269246,0.207534,2.425914,2.108851,1.271546,1.028047,7250.842405,49.970573
2,29-11-2021,4444.528280,57848.773537,623.819158,204.350423,37.188262,12.803239,0.215241,2.406927,2.156258,1.644771,1.192060,7642.250665,51.580068
3,30-11-2021,4637.121617,57011.280474,624.047887,208.705589,38.003426,12.420904,0.214983,2.305691,2.144709,1.762750,1.359894,8002.677220,59.126139
4,01-12-2021,4589.610618,57169.371994,627.529814,229.448724,36.699057,12.293802,0.209487,2.359353,2.141166,1.564412,1.292847,7753.919012,64.317810


In [7]:
#build price percent Changes Graph
dfPercents = dfPrices
dfPercents = dfPercents.set_index('dates')
dfPercents = dfPercents.pct_change()
display(dfPercents.head(5))
#the default y axis label was value so thats why it is being changed below
fig = px.line(dfPercents, labels = {'value': 'Percent Change', 'variable': 'coin'})
py.offline.plot(fig, filename='pricePercentChangesLines.html')

fig.show()


,ethereum,bitcoin,binancecoin,solana,polkadot,flow,dogecoin,audius,fantom,brokoli,unix,wonderland,terra-luna
dates,,,,,,,,,,,,,
27-11-2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28-11-2021,0.050440,0.049589,0.022376,0.043330,0.019830,0.037881,0.015419,0.005855,0.020225,0.000521,0.000000,-0.026135,0.150156
29-11-2021,0.035998,0.010677,0.020232,0.019400,0.043312,0.043523,0.037135,-0.007827,0.022480,0.293520,0.159539,0.053981,0.032209
30-11-2021,0.043333,-0.014477,0.000367,0.021312,0.021920,-0.029862,-0.001199,-0.042060,-0.005356,0.071730,0.140793,0.047162,0.146298
01-12-2021,-0.010246,0.002773,0.005580,0.099389,-0.034322,-0.010233,-0.025565,0.023274,-0.001652,-0.112516,-0.049303,-0.031084,0.087807


In [8]:
dfPrices = dfPrices.set_index('dates')
fig = px.line(dfPrices)
# fig = px.line(dfPercents, labels = {'value': 'Percent Change', 'variable': 'coin'})
py.offline.plot(fig, filename='priceChangesLines.html')

fig.show()
